## Project: prediction of cryptic binding sites

### Introduction

The accurate prediction of protein-ligand binding sites plays a pivotal role in drug discovery, protein engineering, and understanding biological functions at the molecular level. The prediction of binding sites can be carried out either based on the three dimensional structure of a protein or on its sequence. Until recently the structure-based methods have been predominantly used as the binding is governed by structural and physico-chemical complementarity of a protein and binding ligand.

But, structure-based approaches have an inherent problem because they are trained and evaluated on holo structures, i.e., on ligand-bound structures from which ligand was removed, resulting in a clear cavity in the protein structure. This can be problematic as methods trained on such data are biased towards holo-structures and come short when predicting pockets in apo (ligand-free) versions of proteins where the binding site can be obscured. Binding site, which have different conformations in apo and holo forms are called cryptic binding sites. In reality, almost all existing protein-ligand binding site prediction methods are biased toward the discovery of holo-binding sites, as existing benchmarks and training datasets are componsed of holo-structures only. Thus, existing structure-based are not well suited for the detection of cryptic binding sites. Recently, a novel dataset and benchmark containing apo-holo protein pairs with substantial structural rearrangement of binding sites, called [CryptoBench](https://osf.io/pz4a9/), has been introduced.

### Goal

1. Evaluate the structure-based approach for protein-ligand binding site prediction from labs on holo vs. apo versions of proteins from CryptoBench and quantify how the model's performance differs when trained on holo and evaluated on holo (which is how current prediction methods are trained and evaluated) vs when evaluated on apo.
2. Compare the capabilities of the structure based model and protein language models (which are conformation-independent) for the prediction of cryptic binding sites (i.e., evaluation on apo).
3. Try to enrich the protein language model with structural features to improve it's performance. This can be achieved by adding certain properties (such as b-factor, charge, surface exposure, ...) to the NN which takes pLM embeddings on its input.

Regarding the third point, one could, for instance, concatenate the properties with the embeddings and use those as inputs. Another option could be to "inject" these properties somewhere else in the network or use them as a postprocessing. However, any other creative approach will be appreciated.

### Data and model

The train/test splits containing PDB IDs of apo-holo structure pairs (and also corresponding UniProt IDs) can be obtained from the [CryptoBench OSF project site](https://osf.io/pz4a9). The 3D structures can then be downloaded, e.g., via the RCSB PDB's [file download service](https://www.rcsb.org/docs/programmatic-access/file-download-services) and processed with BioPython's [Bio.PDB package](https://biopython.org/docs/1.75/api/Bio.PDB.html) ([here](https://biopython.org/DIST/docs/tutorial/Tutorial.html#sec240) is a short tutorial for working with PDB structures in BioPython).

A lot of code (working with protein language models, the CNN for structure-based binding site prediction, handling protein structures) can be reused from the labs on protein-ligand binding site prediction.

Follows a list of packages that could be used as additional features for cryptic binding site prediction (some might be more suitable than others):
 - [Springcraft](https://springcraft.biotite-python.org/)
 - [MoleculeKit](https://software.acellera.com/moleculekit/index.html)

# PART1 - Predicting binding residues with protein language models

### 1 - Import des bibliothèques

In [1]:
import json
import os
from pathlib import Path
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import csv
from torch import nn
from sklearn.utils import class_weight
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
import urllib.request
import re

torch.manual_seed(0)

In [2]:
#print path
print(os.getcwd())

c:\Users\steve\OneDrive\Desktop\Master BIM\M1\S2\Deep Life\Projet


### 2 - Import des fichiers json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#DATAPATH = "/content/drive/MyDrive/Projet/Data/"
DATAPATH = r'c:\Users\steve\OneDrive\Desktop\Master BIM\M1\S2\Deep Life\Projet\Data'

In [10]:
import pandas as pd 

df = pd.read_csv('Data/apo_train.txt', sep=";", header=None) #0 = nom prot, 1 = chaîne, 2 = sait pas, 3 = labels, 4 = séquence
print(df[4])

0       PSSSMADFRKFFAKAKHIVIISGAGVSAESGVPTFRGAGGYWRKWQ...
1       ALRIDSHQHFWRYRAADYPWIGAGMGVLARDYLPDALHPLMHAQAL...
2       SLRLIATEEAVTFQPVVDALRAHSRTDDASLDMILVRDVYGDEPAR...
3       MLGKVALEEAFALPRHKERTRWWAGLFAIDPDKHAAEINDITEQRI...
4       MRPPLIERYRNLLPVSEKTPVISLLEGSTPLIPLKGPEEARKKGIR...
                              ...                        
1144    DDRELAILATAENLLEDRPLADISVDDLAKGAGISRPTFYFYFPSK...
1145    ALDCRERIEKDLEDLEKELMEMKSIKLSDDEEAVVERALNYRDDSV...
1146    TYRSIGSTAYPTIGVVLLGGIANPVTRTPLHTSAGIAYSDSCGSIR...
1147    LKSVTVSAPSNIAVVKYWGKRGDERLNLPLNNSLSITLDDQLSVIT...
1148    DAVLELPAATFDLPLSLSGGTQTTLRAHAGHWLVIYFYPKDSTPGC...
Name: 4, Length: 1149, dtype: object


In [4]:
def import_json(file_path):
    with open(file_path) as file:
        data = json.load(file)

    return data

# Load the data
# Load the whole dataset
# Load the whole dataset
whole_dataset = import_json(DATAPATH + "\whole_dataset.json")

# Load the train data
train0 = import_json(DATAPATH + "\\train-fold-0.json")
train1 = import_json(DATAPATH + "\\train-fold-1.json")
train2 = import_json(DATAPATH + "\\train-fold-2.json")
train3 = import_json(DATAPATH + "\\train-fold-3.json")
train4 = import_json(DATAPATH + "\\train-fold-4.json")

# Load the test data
test = import_json(DATAPATH + "\\test.json")

In [5]:
#importer les sequences depuis uniprot
#https://rest.uniprot.org/uniprotkb/Q6W7J7.fasta

def import_seqs(dico):
    for k in dico.keys():
        webUrl = urllib.request.urlretrieve(f"https://rest.uniprot.org/uniprotkb/{dico[k][0]['uniprot_id']}.fasta", f"data/sequences/{dico[k][0]['uniprot_id']}.fasta")

In [6]:
if not os.path.exists("Data/sequences"):
    os.makedirs("Data/sequences")
    import_seqs(train0)
    import_seqs(train1)
    import_seqs(test)

In [7]:
def read_fasta(filename):
    with open(filename, 'r') as file :
        seq = ''
        lines = file.readlines()
        for l in lines:
            if l[0] != '>' :
                seq += (l.strip())
    return seq

def create_dataset(dico) :
    datax = {}
    for k in dico.keys() :
        datax[dico[k][0]['uniprot_id']] = []
        datax[dico[k][0]['uniprot_id']].append(read_fasta(f'Data/sequences/{dico[k][0]["uniprot_id"]}.fasta')) #indice 0 séquence
        datax[dico[k][0]['uniprot_id']].append(dico[k][0]['apo_pocket_selection'])                             #indice 1 apo
        datax[dico[k][0]['uniprot_id']].append(dico[k][0]['holo_pocket_selection'])                            #indice 2 holo
    return datax

data_train0 = create_dataset(train0)
data_train1 = create_dataset(train1)
data_test = create_dataset(test)

In [8]:
print(data_train0['Q6W7J7'])
print(data_train0)

['MLEYSELYPIQNEYRMMQSLDGMWKFQFDPEEIGKKSGWENGLPAPVSMPVPSSFADFFTDHKERDYCGDFWYETEFYLPAEWRNKKIWLRFGSITHRGTVYCNGMEITSHEGGFLPVLADISTVAKPGQVNQVVVKINNELNETSLPCGATKILNNGRKLAKPYFDFFNYSGLQRSVWVIALPEESVKDYSVDYELCGTDALVKYEVVTTGEHPVIVRLLDAEGELVAETEGKEGILQVANARLWEVRNAYLYQIVILITDGNGVLDEYREKIGIRTVRIEGTKILLNDRPVYLKGFGKHEDFPILGRGFHWGIVKRDFECLKWTNANCFRTSHYPYAEEWYQFADEEGFLIIDEVPAVGMMRSTRNFVAAGSGNYTYFFEALTVPELLKSHIADTEEMITRDKNHPSVIAWSLFNEPETITDYAYEYFKEVFAAAETYDFQSRPMTGAFEKNSKPELCKCYPLCDFICLNRYYGWYISGGPEIEEAEELFRDEMDRWKAKELNVPFVFTEFGTDTMAGLHKLPSIMWSEEYQKEYLEMNFRVFDSYEFVQGELAWNFADFQTTEGIMRVDGNHKGVFTRDRQPKAAAVVFKDRWEKKNELF', ['C_167', 'C_335', 'C_361', 'C_363', 'C_417', 'C_472', 'C_474', 'C_478', 'C_512', 'C_557', 'C_562', 'C_570', 'C_574', 'C_576'], ['B_167', 'B_335', 'B_361', 'B_363', 'B_417', 'B_472', 'B_474', 'B_478', 'B_512', 'B_557', 'B_562', 'B_570', 'B_574', 'B_576']]
{'Q6W7J7': ['MLEYSELYPIQNEYRMMQSLDGMWKFQFDPEEIGKKSGWENGLPAPVSMPVPSSFADFFTDHKERDYCGDFWYETEFYLPAEWRNKKIWLRFGSITHRGTVYCNGMEITSHEGGFLPVLADIS

In [26]:
from transformers import BertModel, BertTokenizer

tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
model = BertModel.from_pretrained("Rostlab/prot_bert")

#TEST
protein_sequences = ['MKALCLLLLPVLGLLVSSKTLCSMEEAINERIQEVAGSLIFRAISSIGLECQSVTSRGDLATCPRGFAVTGCTCGSACGSWDVRAETTCHCQCAGMDWTGARCCRVQPLEHHHHHH',
      'GSHMSLFDFFKNKGSAATATDRLKLILAKERTLNLPYMEEMRKEIIAVIQKYTKSSDIHFKTLDSNQSVETIEVEIILPR']

# the ProtBert takes aminoacids separated by a space as an input
# this snippet adds a space between each letter of the sequence
spaced_protein_sequences = [seq.replace("", " ")[1: -1] for seq in protein_sequences]

# generate an embedding
output = []
for seq in spaced_protein_sequences:
    encoded_input = tokenizer(seq, return_tensors='pt')

    # WARNING: you need to strip the embedding of the first and last vector, as those are the starting and terminating signs!
    output.append(model(**encoded_input).last_hidden_state.detach().numpy()[0][1:-1])

# SANITY CHECK: length of the sequence matches the number of generated embeddings
for i, seq in zip(output, protein_sequences):
    print(f'Embedding shape: {i.shape}, sequence length: {len(seq)}')

ModuleNotFoundError: No module named 'transformers.utils'

In [10]:
from transformers import EsmModel, EsmTokenizer

tokenizer = EsmTokenizer.from_pretrained("facebook/esm2_t33_650M_UR50D")
model = EsmModel.from_pretrained("facebook/esm2_t33_650M_UR50D")

#TEST
protein_sequences = ['MKALCLLLLPVLGLLVSSKTLCSMEEAINERIQEVAGSLIFRAISSIGLECQSVTSRGDLATCPRGFAVTGCTCGSACGSWDVRAETTCHCQCAGMDWTGARCCRVQPLEHHHHHH',
      'GSHMSLFDFFKNKGSAATATDRLKLILAKERTLNLPYMEEMRKEIIAVIQKYTKSSDIHFKTLDSNQSVETIEVEIILPR']

    # the ProtBert takes aminoacids separated by a space as an input
    # this snippet adds a space between each letter of the sequence
spaced_protein_sequences = [seq.replace("", " ")[1: -1] for seq in protein_sequences]

# generate an embedding
output = []
for seq in spaced_protein_sequences:
    encoded_input = tokenizer(seq, return_tensors='pt')

    # WARNING: you need to strip the embedding of the first and last vector, as those are the starting and terminating signs!
    output.append(model(**encoded_input).last_hidden_state.detach().numpy()[0][1:-1])

# SANITY CHECK: length of the sequence matches the number of generated embeddings
for i, seq in zip(output, protein_sequences):
    print(f'Embedding shape: {i.shape}, sequence length: {len(seq)}')

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding shape: (116, 1280), sequence length: 116
Embedding shape: (80, 1280), sequence length: 80


In [9]:
#Embedding the data_train_sequences

def embed_sequence(data, tokenizer, model):
    embedded_seq = []
    output = []

    for k in data.keys():
        encoded_input = tokenizer(data[k][0], return_tensors='pt')
        output.append(model(**encoded_input).last_hidden_state.detach().numpy()[0][1:-1])
        embedded_seq.append(output)


    apo_labels = [] #on crée une liste vide pour les labels apo
    holo_labels = [] #on crée une liste vide pour les labels holo
    apo_labels = [data[k][1] for k in data.keys()] #on récupère les labels apo
    holo_labels = [data[k][2] for k in data.keys()] #on récupère les labels holo


    filtered_apo_labels = []
    filtered_holo_labels = []

    for k in range(len(apo_labels)):
        filtered_apo_labels.append([])
        for l in range(len(apo_labels[k])):
            filtered_apo_labels[-1].append(int(apo_labels[k][l].split('_')[1]))
    for n in range(len(holo_labels)):
        filtered_holo_labels.append([])
        for h in range(len(holo_labels[n])):
            filtered_holo_labels[-1].append(int(holo_labels[n][h].split('_')[1]))

    bin_apo_label = []
    seq = 0
    for m in data.keys(): #pour chaque sequence (603 pour la 1ère sequence)
        bin_apo_label.append([])
        indice = 0
        #print(m)
        for n in range(len(data[m][0])): #pour chaque aa dans la séquence
            #print('n : numéro de aa',n)

            if indice < len(filtered_apo_labels[seq]) and n == filtered_apo_labels[seq][indice]:
                bin_apo_label[-1].append(1)
                indice += 1
            else:
                bin_apo_label[-1].append(0)
        seq += 1
    bin_holo_label=[]
    return embedded_seq , bin_apo_label , bin_holo_label


In [ ]:
a,b,c=embed_sequence(data_train0, tokenizer, model)

In [1]:
print(a)

NameError: name 'a' is not defined

In [ ]:
import gc

del s_train, Ys_train, train_dataset, Xs_test, Ys_test, test_dataset
del tokenizer, model
del model_0
X
gc.collect()